In [ ]:
#!/user/bin/env python
#--------------------
#@Time  : 2020/4/12 16:22
#@Author: ZHANG YINGJIE
#@File  : network.py


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import slim

#----build ic-CNN net----


def ic_cnn(inputs, scope='iccnn'):
    with tf.variable_scope(scope):
        with tf.variable_scope('lr'):
            lr_column = slim.conv2d(inputs, 64, [3,3], 1, padding='SAME', scope='conv1')
            lr_column = slim.conv2d(lr_column, 64, [3,3], 1, padding='SAME', scope='conv2')
            lr_column = slim.max_pool2d(lr_column, [2,2], 2, scope='maxpool1')
            lr_column = slim.conv2d(lr_column, 128, [3,3], 1, padding='SAME', scope='conv3')
            lr_column = slim.conv2d(lr_column, 128, [3,3], 1, padding='SAME', scope='conv4')
            lr_column = slim.max_pool2d(lr_column, [2,2], 2, scope='maxpool2')
            lr_column = slim.conv2d(lr_column, 256, [3,3], 1, padding='SAME',scope='conv5')
            low_map = lr_column
            lr_column = slim.conv2d(lr_column, 256, [3,3], 1, padding='SAME',scope='conv6')
            lr_column = slim.conv2d(lr_column, 256, [3, 3], 1, padding='SAME', scope='conv7')
            lr_column = slim.conv2d(lr_column, 196, [7, 7], 1, padding='SAME', scope='conv8')
            lr_column = slim.conv2d(lr_column, 96, [5, 5], 1, padding='SAME', scope='conv9')
            lr_column = slim.conv2d(lr_column, 32, [3, 3], 1, padding='SAME', scope='conv10')

        lr_column = slim.conv2d(lr_column, 1, [1,1], 1, padding='SAME', scope='lr_conv11')

        with tf.variable_scope('hr'):
            hr_column = slim.conv2d(inputs, 16, [7,7], 1, padding='SAME', scope='conv1')
            hr_column = slim.max_pool2d(hr_column, [2,2], 2, scope='maxpool1')
            hr_column = slim.conv2d(hr_column, 24, [5,5], 1, padding='SAME', scope='conv2')
            hr_column = slim.max_pool2d(hr_column, [2,2], 2, scope='maxpool2')
            hr_column = slim.conv2d(hr_column, 48, [3,3], 1, padding='SAME', scope='conv3')
            hr_column = slim.conv2d(hr_column, 48, [3,3], 1, padding='SAME', scope='conv4')
            hr_column = slim.conv2d(hr_column, 24, [3, 3], 1, padding='SAME', scope='conv5')
            high_map = hr_column
            three_map = tf.concat([low_map, lr_column, high_map], axis=-1)
            hr_column = slim.conv2d(three_map, 196, [7, 7], 1, padding='SAME', scope='conv6')
            hr_column = slim.conv2d(hr_column, 96, [5, 5], 1, padding='SAME', scope='conv7')
            
            strides = [1, 2, 2, 1]
            in_shape_tensor = tf.shape(hr_column)
            output_shape = [in_shape_tensor[0], 2 * in_shape_tensor[1], 2 * in_shape_tensor[2], 96]
            filter = tf.constant(0.25, shape=[2, 2, 96, 96])
            hr_column = tf.nn.conv2d_transpose(hr_column, filter, output_shape, strides)
            
            hr_column = slim.conv2d(hr_column, 32, [3, 3], 1, padding='SAME', scope='conv8')
            
            in_shape_tensor = tf.shape(hr_column)
            filter = tf.constant(0.25, shape=[2,2,32,32])
            output_shape = [in_shape_tensor[0], 2 * in_shape_tensor[1], 2 * in_shape_tensor[2], 32]
            hr_column = tf.nn.conv2d_transpose(hr_column, filter, output_shape, strides)

        hr_column = slim.conv2d(hr_column, 1, [1,1], 1, padding='SAME', scope='hr_conv9')

    return hr_column, lr_column

